<h1 align="center">ECG signal denoising using discrete wavelet transform (DWT) signal  processing technique in python using PyWavelet </h1> 

In [ ]:
# Welcome to the ECG Signal processing, please note these program has two parts 
# Part one is used to process noisy ECG signals, Please run line by line till the end of the first part
# Part two is for ECG signals without noise
# The output file is saved as data1.csv in the folder, reselect this file for classification in the java app


## PART ONE - Noisy ECG Signal Processing - RUN LINE by LINE till the END

In [ ]:
import pywt
import pandas as pd
import numpy as np
import heartpy as hp
import matplotlib.pyplot as plt
import scipy.signal as signal


In [ ]:

print('Enter ECG Data to denoise')
file = input("Input File path: ")

data = pd.read_csv(file)

data.head()

ecg = data.iloc[:,1] # select the second column which is the voltage

print(ecg[0:20])

fs = 360
t = np.arange(len(data))/fs #time of the data, convert sampling frequency to time domain

In [ ]:
coeffs = pywt.wavedec(ecg, 'db4', level=4) # perform 4 level decomp with the sym4 wavelet

In [ ]:
cA4, cD4, cD3, cD2, cD1 = coeffs # wave decompose returns a data array [cA_n, cD_n, cD_n-1, …, cD2, cD1]

In [ ]:
print(cD1)

In [ ]:
rdata = pywt.idwt(cA4, cD4, 'sym4') # reconstructed ECG data without noise 

In [ ]:
ecg1 = ecg[0:400] # plot for 500 data-points 
t = t[0:400]
cD1 = cD1[0:400]
cD2 = cD2[0:400]
cD3 = cD3[0:400]
cD4 = cD4[0:400]
rdata = rdata[0:400]



plt.figure(figsize=(20,20))

plt.subplot(6,1,1)
plt.plot(t,ecg1, color='r')
plt.ylabel('Amplitude')
plt.title('Original ECG Data')

plt.subplot(6,1,2)
plt.plot(t,cD1, color='k')
plt.ylabel('Amplitude')
plt.title('level 1 decomposition')

plt.subplot(6,1,3)
plt.plot(t,cD2, color='y')
plt.ylabel('Amplitude')
plt.title('level 2 decomposition')

plt.subplot(6,1,4)
plt.plot(t,cD3, color='b')
plt.ylabel('Amplitude')
plt.title('level 3 decomposition')

plt.subplot(6,1,5)
plt.plot(t,cD4, color='g')
plt.ylabel('Amplitude')
plt.title('level 4 decomposition')

plt.subplot(6,1,6)
plt.plot(t,rdata, color='m')

plt.ylabel('Amplitude')
plt.xlabel('Time')
plt.title('Reconstructed Denoised ECG Data')
plt.savefig('denoised_ECG.png', dpi=100)


In [ ]:
from scipy.signal import resample
#resample the data. Usually 2, 4, or 6 times is enough depending on original sampling rate
resampled_data = resample(rdata, len(rdata) * 4)


plt.figure(figsize=(12,3))
plt.plot(resampled_data[0:400])
plt.show

wd, m = hp.process(hp.scale_data(resampled_data), fs)

plt.figure(figsize=(100,20))
hp.plotter(wd, m)

for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))
        
#plot poincare
hp.plot_poincare(wd, m)


In [ ]:
my_data = rdata[0:186]
j = signal.resample(my_data,186)

In [ ]:
jsquared =j*j# square the resampled data
jrnd = np.round(jsquared,3)# round data to 3 decimal placess
jrnd1 = np.concatenate((jrnd,"?"),axis=None)#for csv generation for classifying with java application
print(jrnd1)
df = pd.DataFrame(jrnd1).T
df.to_csv('data1.csv',header=False)

## ## PART TWO - Noiseless ECG Signal Processing - RUN LINE by LINE till the END

In [ ]:
import pywt
import pandas as pd
import numpy as np
import heartpy as hp
import matplotlib.pyplot as plt
import scipy.signal as signal

In [ ]:
print('Enter ECG Data to denoise')
file = input("Input File path: ")

data = pd.read_csv(file)

data.head()

ecg = data.iloc[:,1] # select the second column which is the voltage

print(ecg[0:20])

fs = 360 # be sure to input the right sampling frequency
t = np.arange(len(data))/fs #time of the data, convert sampling frequency to time domain

In [ ]:
ecg1 = ecg[0:400] # plot for 500 data-points 
t = t[0:400]

In [ ]:
plt.figure(figsize=(100,20))
plt.plot(t,ecg1, color='r')
plt.ylabel('Amplitude')
plt.title('Original ECG Data')

In [ ]:
from scipy.signal import resample
#resample the data. Usually 2, 4, or 6 times is enough depending on original sampling rate
resampled_data = resample(ecg, len(ecg) * 4)


plt.figure(figsize=(12,3))
plt.plot(resampled_data[0:400])
plt.show

wd, m = hp.process(hp.scale_data(resampled_data), fs)

plt.figure(figsize=(100,20))
hp.plotter(wd, m)

for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))
        
#plot poincare
hp.plot_poincare(wd, m)

In [ ]:
my_data = ecg[0:186]
j = signal.resample(my_data,186) #resample data to process ECG data

In [ ]:
jsquared =j*j# square the resampled data
jrnd = np.round(jsquared,3)# round data to 3 decimal placess
jrnd1 = np.concatenate((jrnd,"?"),axis=None)#for csv generation for classifying with java application
print(jrnd1)
df = pd.DataFrame(jrnd1).T
df.to_csv('data1.csv',header=False) # output file as csv